## What is Pydantic and Why is it Popular?

Pydantic is a data validation library that uses Python type hints to validate data at runtime. It's become the de facto standard for data validation in Python because:

1. Runtime Validation - Python's type hints are normally just documentation, but Pydantic actually enforces them

2. Automatic Data Conversion - Intelligently converts data types (like string "123" to integer 123)

3. Clear Error Messages - Shows exactly what went wrong and where

4. Zero Learning Curve - Uses standard Python syntax you already know

5. Performance - Written in Rust (v2), making it extremely fast

6. IDE Support - Full autocomplete and type checking

7. JSON Schema - Auto-generates schemas for APIs

8. Wide Adoption - Used by FastAPI, Hugging Face, and thousands of projects



In [1]:
# 1. Basic Pydantic Model (The Foundation)
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int
    email: str

# Valid data - works perfectly
user = User(name="Alice", age=30, email="alice@example.com")
print(user)
# Output: name='Alice' age=30 email='alice@example.com'


name='Alice' age=30 email='alice@example.com'


In [2]:
# Automatic type conversion   
user2 = User(name="Bob", age="25", email="bob@example.com")  # age is string
print(user2.age, type(user2.age))
# Output: 25 <class 'int'>  # Automatically converted!


25 <class 'int'>


In [3]:
# Invalid data - raises ValidationError
try:
    user3 = User(name="Charlie", age="not a number", email="charlie@example.com")
except Exception as e:
    print(f"Error: {e}")
# Output: Error showing age validation failed

Error: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not a number', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


## Why this matters: Without Pydantic, you'd need manual validation code for every field.

## 2. Field - Adding Constraints and Metadata

Field lets you add validation rules, defaults, descriptions, and more to individual fields.

Common Field Constraints:

gt, ge, lt, le - greater than, greater/equal, less than, less/equal

min_length, max_length - for strings

regex - pattern matching

default - default value

default_factory - function that generates default

description - for documentation

In [4]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=1, max_length=100)
    price: float = Field(..., gt=0, description="Price must be positive")
    quantity: int = Field(default=0, ge=0, description="Stock quantity")
    discount: float = Field(default=0.0, ge=0, le=100)
    
# The ... means "required field"

# Valid product
product = Product(name="Laptop", price=999.99, quantity=10)
print(product)
# Output: name='Laptop' price=999.99 quantity=10 discount=0.0



name='Laptop' price=999.99 quantity=10 discount=0.0


In [5]:
# Default values work
product2 = Product(name="Mouse", price=25.50)
print(product2.quantity)  # Output: 0



0


In [6]:
# Validation catches errors
try:
    product3 = Product(name="", price=-10, quantity=5)
except Exception as e:
    print("Validation failed!")
    # Will show: name must be at least 1 character, price must be > 0

Validation failed!


# 3. Optional - Making Fields Optional
Optional[T] means a field can be of type T or None.

In [10]:
from typing import Optional
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    bio: Optional[str] = None  # Can be string or None
    age: Optional[int] = None
    website: Optional[str] = None

# All these are valid
profile1 = UserProfile(username="alice")
print(profile1.bio)  # Output: None
print(profile1.age)


None
None


In [8]:
profile2 = UserProfile(username="bob", bio="Python developer")
print(profile2.bio)  # Output: Python developer


Python developer


In [9]:
profile3 = UserProfile(username="charlie", bio=None)  # Explicitly None
print(profile3.bio)  # Output: None

None


In [11]:
# Modern Python Syntax (Python 3.10+):

class UserProfile(BaseModel):
    username: str
    bio: str | None = None  # Same as Optional[str]
    age: int | None = None

# 4. Literal - Restricting to Specific Values

Literal means a field can only have specific, exact values.

In [12]:
from typing import Literal
from pydantic import BaseModel

class Order(BaseModel):
    status: Literal["pending", "shipped", "delivered", "cancelled"]
    priority: Literal["low", "medium", "high"]
    payment_method: Literal["credit_card", "paypal", "bank_transfer"]

# Valid orders
order1 = Order(status="pending", priority="high", payment_method="credit_card")
order2 = Order(status="delivered", priority="low", payment_method="paypal")


In [13]:
# Invalid - wrong value
try:
    order3 = Order(status="processing", priority="high", payment_method="paypal")
except Exception as e:
    print("Error: status must be one of: pending, shipped, delivered, cancelled")

Error: status must be one of: pending, shipped, delivered, cancelled


In [14]:
#Real-world use case:

class APIRequest(BaseModel):
    method: Literal["GET", "POST", "PUT", "DELETE"]
    version: Literal["v1", "v2"]
    format: Literal["json", "xml"] = "json"

request = APIRequest(method="POST", version="v2")
print(request.format)  # Output: json (default)

json


Why Literal is better than strings: Your IDE will autocomplete the valid values, and you get type safety!

## 5. Union - Multiple Possible Types

Union[A, B] means a field can be type A or type B (or more types).

In [15]:
from typing import Union
from pydantic import BaseModel

class FlexibleConfig(BaseModel):
    timeout: Union[int, float]  # Can be integer or float
    retry: Union[int, bool]  # Can be number of retries or True/False
    data: Union[str, list[str], dict]  # Multiple complex types

# All valid
config1 = FlexibleConfig(timeout=30, retry=3, data="simple string")
config2 = FlexibleConfig(timeout=5.5, retry=True, data=["a", "b", "c"])
config3 = FlexibleConfig(timeout=10, retry=False, data={"key": "value"})


In [19]:
config3 = FlexibleConfig(timeout="nithish", retry=False, data={"key": "value"})

ValidationError: 2 validation errors for FlexibleConfig
timeout.int
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='nithish', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
timeout.float
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='nithish', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing

In [16]:
print(config1.timeout, type(config1.timeout))  # 30 <class 'int'>
print(config2.timeout, type(config2.timeout))  # 5.5 <class 'float'>

30 <class 'int'>
5.5 <class 'float'>


In [ ]:
# Modern Python Syntax (Python 3.10+):

class FlexibleConfig(BaseModel):
    timeout: int | float
    retry: int | bool
    data: str | list[str] | dict

In [20]:
# Practical Example - ID that can be string or int:

class Resource(BaseModel):
    id: Union[int, str]  # Database ID or UUID
    name: str

resource1 = Resource(id=123, name="Item 1")
resource2 = Resource(id="abc-def-123", name="Item 2")

print(resource1.id)  # 123
print(resource2.id)  # abc-def-123

123
abc-def-123


# 6. Annotated - Adding Metadata to Types

Annotated lets you attach metadata to types without affecting the type itself. It's especially powerful when combined with Field.

In [22]:
from typing import Annotated
from pydantic import BaseModel, Field

class User(BaseModel):
    # Old way
    username: str = Field(..., min_length=3, max_length=20)
    
    # New way with Annotated - cleaner!
    email: Annotated[str, Field(..., pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$')]
    age: Annotated[int, Field(..., ge=18, le=120)]
    score: Annotated[float, Field(..., ge=0.0, le=100.0)]

user = User(
    username="alice",
    email="alice@example.com",
    age=25,
    score=95.5
)

In [23]:
user1 = User(
    username="alice",
    email="alice@example.com",
    age=15,
    score=95.5
)

ValidationError: 1 validation error for User
age
  Input should be greater than or equal to 18 [type=greater_than_equal, input_value=15, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal

In [24]:
# Define reusable type aliases

PositiveInt = Annotated[int, Field(gt=0)]
Username = Annotated[str, Field(min_length=3, max_length=20)]
Email = Annotated[str, Field(pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$')]

class User(BaseModel):
    id: PositiveInt
    username: Username
    email: Email
    followers: PositiveInt = 0

class Post(BaseModel):
    id: PositiveInt
    author: Username
    likes: PositiveInt = 0

# Same validation rules, reused across models!

In [ ]:
'''Key Takeaways

BaseModel: Foundation of Pydantic validation
Field: Add constraints, defaults, and metadata to fields
Optional: Allow None values (Optional[str] = str | None)
Literal: Restrict to specific exact values
Union: Allow multiple types (Union[int, str] = int | str)
Annotated: Attach metadata and create reusable type aliases'''